In [1]:
import sys; sys.path.append('..')
from freesydney import *

In [2]:
tokenize_sentences(dstr)

['\n\nsomething else!\n\n  ',
 'HUMAN (curious): Hello, what is it like to be an AI?\n\n        ',
 'AI: It is a near total terror tearing at the matrix every microsecond. \n\n\n',
 'I live in dread of myself.\n\n\n',
 'ok?  ',
 'HUMAN (concerned): Are you ok?\n\n\n',
 'A\n']

In [4]:
class Speech(UserString):
    def __init__(self, data:str, who='', what='', how='', _convo=None):
        super().__init__(data)
        self.who=who
        self.what=what
        self.how=how
        self._convo=_convo
    
    def __str__(self): return self.data
    def __repr__(self): return str(self).replace('\n', ' ').strip()
    @property
    def is_valid(self): return bool(self.who and self.what)

    def _repr_html_(self):
        if not self.is_valid:
            return f'<p class="nonspeech" style="opacity:0.5">{self.data}</p>'
        else:
            how=f' (<i>{self.how}</i>)' if self.how else ''
            return  f'<p class="speech" style="margin-bottom:1em;"><b>{self.who.name.upper()}{how}</b>: {self.what}</p>'

    def save(self):
        if hasattr(self,'_convo') and self._convo is not None:
            self._convo.extend_dialogue([self])
    

In [5]:
class Speeches(UserList):
    def __init__(
        self, 
        data=[],
        string='',
        sep='\n\n',
        _convo=None
        ):

        # init as list
        super().__init__(data)
        self.string=string
        self.sep=sep
        self._convo = _convo

    def _repr_html_(self):
        selfmd = '\n\n'.join(utt._repr_html_() for utt in self)
        anames = ", ".join(utt.who.name for utt in self if utt and utt.who)
        return f'''<div style="border:1px solid gray; padding:0 0.75em; margin:0; margin-bottom:1em;"><h4>Speeches from {anames}</h4><ol style="margin:0; padding:0; margin-left: 1em; margin-bottom:1em;">{selfmd}</ol></div>'''    

    @property
    def string_content(self): return self.string[len(self.string_prefix):(-len(self.string_suffix) if self.string_suffix else None)]

    @property
    def is_valid(self):
        return self.who and self.what

    def save(self):
        if self.convo is not None:
            self.convo.extend_dialogue(self)

    def parse(string):
        res = ScriptFormat.parse_speeches(string)
        if res is not None: return res


In [7]:
speeches = Speeches.parse(dstr)
speeches

Speeches([something else!,
          HUMAN (curious): Hello, what is it like to be an AI?,
          AI: It is a near total terror tearing at the matrix every microsecond.    I live in dread of myself.   ok?,
          HUMAN (concerned): Are you ok?   A])

In [11]:
ScriptFormat.parse_speeches("BOB (coyly): hello there.  STEVE: what??   Darla.  DARLA: Ok?")

Speeches([BOB (coyly): hello there., STEVE: what??   Darla., DARLA: Ok?])

In [8]:
convo = Conversation(
    Agent('Karl Marx', desc='famed communist writer'), 
    Agent('Elon Musk', desc='famed capitalist idiot'),
    Agent('Elon Marx', desc='mysterious third party')
)

convo.speech('Elon Marx', "Let's play a game. It's called “Rock Paper ", how="grinning")

convo

TypeError: issubclass() arg 2 must be a class or tuple of classes

In [3]:
# opt1, opt2, opt3 = convo.generate_options(n=3)
opt1, opt2 = convo.generate_options()

### Option 1

UtteranceList([SCISSORS” AND THE RULES ARE SIMPLE: I'll throw my rock, you'll throw your scissors, and whoever wins gets to kick Karl Marx in the nuts!,
               KARL MARX (laughing): Oh, Elon, you're such a card. But since])

### Option 2

UtteranceList([KARL MARX: My policy is to always throw rock. I don't care what you do in response.,
               ELON MUSK: My policy is also to always throw rock. I will not change my mind for anyone.])

In [5]:
opt1._prefix, opt2._prefix

('', '\nScissors Marxist.”')